In [114]:
EXP_NAME = "exp001"
MODEL_NAME = "distilbert-base-uncased"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME}"
LOG_PATH = f"pll_data_detection/log/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"pll_data_detection/trained_models/{EXP_NAME}"
DEBUG = False
UPLOAD_DATA = True

# Install

In [111]:
%pip install polars
%pip install transformers==4.37.2
%pip install datasets==2.16.1
%pip install evaluate==0.4.1
%pip install seqeval==1.2.2
%pip install accelerate
%pip install python-dotenv
%pip install kaggle

# formatter
%pip install black isort

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.5-py3-none-any.whl size=111935 sha256=881a97167a9429cf1b599992c2821ea4659e64ca6c77fb6060dfa2c5d17ea58f
  Stored in directory: /root/.cache/pip/wheels/be/79/f7/3e60835417ed066f60bc88cdd67a14d77eea41e391c0af2c3c
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Import

In [81]:
import os

import evaluate
import numpy as np
import polars as pl
import torch
import wandb
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TokenClassificationPipeline,
    Trainer,
    TrainingArguments,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [82]:
import transformers

assert transformers.__version__ == "4.37.2"

In [83]:
import datasets

assert datasets.__version__ == "2.16.1"

In [84]:
import evaluate

assert evaluate.__version__ == "0.4.1"

# Wandb

In [85]:
from dotenv import load_dotenv

load_dotenv("pll_data_detection/.env")

wandb.login(key=os.environ["WANDB_API_KEY"])
wandb.init(project="pll", name=EXP_NAME)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


eval/accuracy,▁██
eval/f1,▁▁▁
eval/loss,█▃▁
eval/precision,▁▁▁
eval/recall,▁▁▁
eval/runtime,▁▂█
eval/samples_per_second,█▇▁
eval/steps_per_second,█▇▁
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁


# Check Environment

In [86]:
!python --version

Python 3.9.16


In [87]:
!nvidia-smi

Thu Feb  8 02:17:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 28%   37C    P8     7W / 180W |   6958MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Load

In [88]:
# データ確認用
train = pl.read_json("pll_data_detection/train.json")
train.head()

document,full_text,tokens,trailing_whitespace,labels
i64,str,list[str],list[bool],list[str]
7,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
10,"""Diego Estrada …","[""Diego"", ""Estrada"", … "" ""]","[true, false, … false]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]"
16,"""Reporting proc…","[""Reporting"", ""process"", … "" ""]","[true, false, … false]","[""O"", ""O"", … ""O""]"
20,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]","[""O"", ""O"", … ""O""]"
56,"""Assignment: V…","[""Assignment"", "":"", … "" ""]","[false, false, … false]","[""O"", ""O"", … ""O""]"


In [89]:
# データ確認用
test = pl.read_json("pll_data_detection/test.json")
test

document,full_text,tokens,trailing_whitespace
i64,str,list[str],list[bool]
7,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]"
10,"""Diego Estrada …","[""Diego"", ""Estrada"", … "" ""]","[true, false, … false]"
16,"""Reporting proc…","[""Reporting"", ""process"", … "" ""]","[true, false, … false]"
20,"""Design Thinkin…","[""Design"", ""Thinking"", … "" ""]","[true, true, … false]"
56,"""Assignment: V…","[""Assignment"", "":"", … "" ""]","[false, false, … false]"
86,"""Cheese Startup…","[""Cheese"", ""Startup"", … "" ""]","[true, true, … false]"
93,"""Silvia Villalo…","[""Silvia"", ""Villalobos"", … "" ""]","[true, false, … false]"
104,"""Storytelling …","[""Storytelling"", "" "", … "" ""]","[true, false, … false]"
112,"""Reflection – L…","[""Reflection"", ""–"", … "" ""]","[true, true, … false]"


In [90]:
train_dataset = load_dataset(
    "json", data_files={"train": "pll_data_detection/train.json"}, split="train"
)
test_dataset = load_dataset(
    "json", data_files={"test": "pll_data_detection/test.json"}, split="test"
)

In [91]:
# debug
if DEBUG:
    train_dataset = train_dataset.select(range(100))

In [92]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [93]:
# TODO: 512?が上限であるのに対し、NER対象のテキストが512よりも長い場合が多いため、適したモデルにする
# DeBERTaあたりは、512を超えるテキストに対応していたような
example = train_dataset[0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

Token indices sequence length is longer than the specified maximum sequence length for this model (748 > 512). Running this sequence through the model will result in indexing errors


['[CLS]',
 'design',
 'thinking',
 'for',
 'innovation',
 'reflex',
 '##ion',
 '-',
 'av',
 '##ril']

In [94]:
# labelを変換する
id2label = {
    0: "O",
    1: "B-NAME_STUDENT",
    2: "I-NAME_STUDENT",
    3: "B-EMAIL",
    4: "I-EMAIL",
    5: "B-USERNAME",
    6: "I-USERNAME",
    7: "B-ID_NUM",
    8: "I-ID_NUM",
    9: "B-PHONE_NUM",
    10: "I-PHONE_NUM",
    11: "B-URL_PERSONAL",
    12: "I-URL_PERSONAL",
    13: "B-STREET_ADDRESS",
    14: "I-STREET_ADDRESS",
}

label2id = {v: k for k, v in id2label.items()}


def label2id_func(example):
    example["labels"] = [label2id[tag] for tag in example["labels"]]
    return example


labele2id_train_dataset = train_dataset.map(label2id_func)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [95]:
label_list = list(label2id.keys())
label_list

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-EMAIL',
 'I-EMAIL',
 'B-USERNAME',
 'I-USERNAME',
 'B-ID_NUM',
 'I-ID_NUM',
 'B-PHONE_NUM',
 'I-PHONE_NUM',
 'B-URL_PERSONAL',
 'I-URL_PERSONAL',
 'B-STREET_ADDRESS',
 'I-STREET_ADDRESS']

In [96]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["labels"]):
        # トークン化されたシーケンス内の各トークンが元のテキスト内のどの単語に対応するかを示すIDを提供します。
        # 例えば、元のテキストが "Hello, world!" で、トークン化されたシーケンスが ["Hello", ",", "world", "!"] の場合
        # `word_ids`メソッドは [0, None, 1, None] を返します。
        # これは、"Hello" が最初の単語（インデックス0）、"," が単語に属さない（None）、"world" が2番目の単語（インデックス1）、"!" が単語に属さない（None）ことを示します。
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif (
                word_idx != previous_word_idx
            ):  # subwordの場合、同じword_idxが連続している。最初のtoken以外は-100にする
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_train_dataset = labele2id_train_dataset.map(
    tokenize_and_align_labels, batched=True
)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [97]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [98]:
seqeval = evaluate.load("seqeval")

In [99]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [100]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
# evalデータの用意
tokenized_train_valid_dataset = tokenized_train_dataset.train_test_split(test_size=0.2)

In [102]:
training_args = TrainingArguments(
    output_dir=LOG_PATH,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # num_train_epochs=2,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model="f1",  # add
    greater_is_better=True,  # add
    warmup_ratio=0.1,  # add
    lr_scheduler_type="cosine",  # add
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_valid_dataset["train"],
    eval_dataset=tokenized_train_valid_dataset["test"],  # 名前はtestだが、実際はvalid
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [103]:
# モデルの保存
trainer.train()
trainer.save_model(MODEL_OUTPUT_PATH)

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.002720,0.232283,0.228682,0.230469,0.999541
2,0.193800,0.001771,0.748768,0.589147,0.659436,0.999749
3,0.001600,0.001651,0.723214,0.627907,0.672199,0.999757


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory pll_data_detection/log/exp001/checkpoint-341 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Checkpoint destination directory pll_data_detection/log/exp001/checkpoint-682 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetric

# Data Upload

In [105]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system("cp /notebooks/pll_data_detection/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [115]:
from kaggle.api.kaggle_api_extended import KaggleApi
import json


def dataset_create_new(dataset_name: str, upload_dir: str):
    if "_" in dataset_name:
        raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata["id"] = f"sinchir0/{dataset_name}"
    dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
    dataset_metadata["title"] = dataset_name
    with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")


if (not DEBUG) and UPLOAD_DATA:
    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:exp001-distilbert-base-uncased, output_dir:pll_data_detection/trained_models/exp001
Starting upload for file pytorch_model.bin


100%|██████████| 253M/253M [00:03<00:00, 75.0MB/s] 


Upload successful: pytorch_model.bin (253MB)
Starting upload for file training_args.bin


100%|██████████| 4.17k/4.17k [00:00<00:00, 6.64kB/s]


Upload successful: training_args.bin (4KB)
Starting upload for file vocab.txt


...

Upload successful: model.safetensors (253MB)
Starting upload for file special_tokens_map.json


100%|██████████| 125/125 [00:00<00:00, 263B/s]


Upload successful: special_tokens_map.json (125B)
Starting upload for file .ipynb_checkpoints.tar


100%|██████████| 10.0k/10.0k [00:00<00:00, 18.9kB/s]


Upload successful: .ipynb_checkpoints.tar (10KB)
